In [35]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image
from sklearn import preprocessing
import requests
import pickle
from io import BytesIO

from sklearn.mixture import GaussianMixture
from sklearn.neighbors import KernelDensity


# Acceptance rates

In [165]:
data = pickle.load(file('faces_FINAL_chosen_and_not_chosen.pickle'))
data.chosen.fillna(False, inplace=True)
data.head()

,category,seed,latent,node_id,chosen
0,masculine,10,"[0.285737406245, -0.312297149698, -0.379479716...",1,False
1,happy,0,"[-1.78785446139, 0.705027015333, -1.1080687416...",2,False
2,happy,0,"[-0.0276354127872, 0.16397054907, 0.0467878324...",2,True
3,sad,9,"[-1.3123048262, 0.837042955277, -0.26302994423...",3,False
4,sad,9,"[-0.769067429448, 0.364602031936, -0.357401259...",3,True


In [176]:
categories = np.unique(data.category)
print categories

[u'feminine' u'happy' u'masculine' u'sad']


In [180]:
[np.mean(data[data.category == CATEGORY].chosen[1::2]) for CATEGORY in categories]

[0.51842751842751844,
 0.53105196451204051,
 0.45006165228113443,
 0.55343511450381677]

In [181]:
[np.mean(data[data.seed == SEED].chosen[1::2]) for SEED in range(16)]

[0.52093023255813953,
 0.59276018099547512,
 0.5643564356435643,
 0.56185567010309279,
 0.52849740932642486,
 0.50555555555555554,
 0.551219512195122,
 0.48936170212765956,
 0.52427184466019416,
 0.517948717948718,
 0.44502617801047123,
 0.470873786407767,
 0.55428571428571427,
 0.58947368421052626,
 0.53051643192488263,
 0.54867256637168138]

# Latent vectors to render

In [142]:
data = pickle.load(file('faces_FINAL_chosen_and_not_chosen.pickle'))
data.chosen.fillna(False, inplace=True)
data = data[data.chosen == True] # only consider chosen nodes
data.head() # every 64 rows is a trial/participant

,category,seed,latent,node_id,chosen
2,happy,0,"[-0.0276354127872, 0.16397054907, 0.0467878324...",2,True
4,sad,9,"[-0.769067429448, 0.364602031936, -0.357401259...",3,True
6,feminine,3,"[0.249914258085, 0.313559376326, -0.8232068547...",4,True
8,feminine,3,"[-1.49686590711, 0.34375606641, 0.330541333266...",5,True
10,feminine,15,"[0.77797707449, -0.346722574145, -0.4472911447...",6,True


In [189]:
## Multivariate Gaussian Sampling

CATEGORY = 'happy'

X = np.stack(data[data.category == CATEGORY].latent)

cov = np.cov(X.T)
# cov = np.diag(np.var(X, axis=0)) # assumes no covariance
mean = np.mean(X, axis=0)

# kde = KernelDensity().fit(X)
# print kde.sample(n_samples=20).tolist()

# copy output into deep_mcmcp_v2/static/scripts/vectors.js
# and open deep_mcmcp_v2/templates/viz.html
print np.random.multivariate_normal(mean=mean, cov=cov, size=20).tolist()

In [188]:
# Gaussian Mixture Model Sampling

CATEGORY = 'feminine'
COVARIANCE_TYPE = 'spherical'
N_COMPONENTS = 10

X = np.stack(data[data.category == CATEGORY].latent)

# copy output into deep_mcmcp_v2/static/scripts/vectors.js
# and open deep_mcmcp_v2/templates/viz.html
gmm = GaussianMixture(n_components=N_COMPONENTS, covariance_type=COVARIANCE_TYPE).fit(X)
print gmm.sample(n_samples=20)[0].tolist()

[[-0.8737955830094957, 1.1633594292433578, -0.06508900809316864, 2.2008321264297868, 1.405948557100101, -0.25842616899194915, 0.4618514801315675, 0.6837351012636187, 0.7708534563134654, -1.919245046743941, -0.8355209201929836, -0.7775861499424113, 0.4333074762458037, -1.9687815928576329, 1.0116213099493576, 1.0136788147008038, -1.9175336084719365, 1.0247481845391102, -0.3607678038136389, 0.9248806563496244, 0.8069705420436425, 1.3952434730254746, 1.5826481419208802, 0.11705646717007398, 0.5274933603274765, -0.12791175788040876, -0.48772613860139047, 1.1166005628665947, -1.6840760107132073, 0.353318605118483, -0.17077246984484845, -0.5797930449165866, -1.3875984984883847, 0.6654883139732668, 1.0090777328501992, 0.23948094195549907, 0.2575818737832022, -1.2594056631545087, -1.6530310875613805, -0.6398485284982424, 0.8126105144995125, 0.22401701160973525, -1.095565661475611, 1.9681547593012498, -1.9775041352085097, -1.8204227220293585, 0.8490787243201066, -1.0187587970530467, 1.7182761065

In [108]:
# Category means

CATEGORY = 'happy'
np.mean(np.stack(data[data.category == CATEGORY].latent), axis=0)

In [106]:
# Seeds

for SEED in range(16):
    print repr(list(list(data[data.seed == SEED].latent)[0]))

In [107]:
# Print means for each seed

print np.stack(np.mean(data[data.seed == SEED].latent, axis=0) for SEED in range(16)).tolist()